# Notes and excercises from *Structure and Interpretation of Computer Programs*

The book is [available online on MIT's site](https://mitpress.mit.edu/sites/default/files/sicp/full-text/book/book.html), there are also [recorded lectures on MIT's OCW](https://www.youtube.com/playlist?list=PLE18841CABEA24090).

## 1. Building Abstractions with Procedures

In [1]:
(define π 3.14)

π

3.14

In [2]:
(define (square x) (* x x))

(assert = (square 2) 4)
(assert = (square 3) 9)

ok

In [3]:
(define (circle-area r)
  (* π (square r)))

(assert = (circle-area 1) π)
(assert = (circle-area 2) 12.56)

ok

In [19]:
(define ε 0.001)

(define (approx x y)
  (< (abs (- x y)) ε))

(assert = (approx 1 1) #t)
(assert = (approx 1.0000001 1) #t)

ok

In [26]:
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x)
                 x)))

(define (improve guess x)
  (average guess (/ x guess)))

(define (average x y)
  (/ (+ x y) 2))

(define (good-enough? guess x)
  (approx (square guess) x))

(define (sqrt x)
  (sqrt-iter 1 x))

(assert approx (sqrt 4) 2)
(assert approx (sqrt 9) 3)

ok

**Exercise 1.6.**  Alyssa P. Hacker doesn't see why if needs to be provided as a special form. ``Why can't I just define it as an ordinary procedure in terms of `cond`?'' she asks. Alyssa's friend Eva Lu Ator claims this can indeed be done, and she defines a new version of `if`:

In [28]:
(define (new-if predicate then-clause else-clause)
  (cond (predicate then-clause)
        (else else-clause)))

(assert = (new-if (= 2 3) 0 5) 5)
(assert = (new-if (= 1 1) 0 5) 0)

ok

In [35]:
(define (sqrt-iter-2 guess x)
  (new-if (good-enough? guess x)
          guess
          (sqrt-iter-2 (improve guess x)
                       x)))

;; (sqrt-iter-2 1 2)

**Exercise 1.7.**  The `good-enough?` test used in computing square roots will not be very effective for finding the square roots of very small numbers. Also, in real computers, arithmetic operations are almost always performed with limited precision. This makes our test inadequate for very large numbers. Explain these statements, with examples showing how the test fails for small and large numbers. An alternative strategy for implementing `good-enough?` is to watch how guess changes from one iteration to the next and to stop when the change is a very small fraction of the `guess`. Design a square-root procedure that uses this kind of end test. Does this work better for small and large numbers? 